# HDF5 and ontologies

HDF5 itself is considered self-describing due to the ability to store metadata (attributes) with raw data. However, this is only the prerequisite. Especially, achieving (easy) re-usability, requires standardized metadata, which is publically defined and accessible. This means, that data must be describable with persistent identifiers, as known from linked data solutions.

One solution of describing data is using controlled vocabularies or even better ontologies. In fact, an [ontology exists]("http://purl.allotrope.org/ontologies/hdf5/1.8#"), which allows describing the structural content of an HDF5 file (groups, datasets, attributes, properties etc.). The ``h5rdmtoolbox`` has implemented a conversion function, translating an HDF5 into a JSON-LD file. This is outlined here.

In [1]:
import h5rdmtoolbox as h5tbx

Failed to import module h5tbx


Let's create a sample HDF5 file first:

In [15]:
from ontolutils import namespacelib
with h5tbx.File() as h5:
    ds = h5.create_dataset('test', data=1,
                           attrs={'units': 'm/s',
                                  'a number': 3})
    ds.attrs['dsref'] = ds
    d = h5tbx.jsonld.dumpd(
        h5,
        compact=True,
        context={'units': namespacelib.M4I.hasUnit,
                 'local': '_:'}
    )
    h5.dump()
import pprint
pprint.pprint(d)

{'@context': {'foaf': 'http://xmlns.com/foaf/0.1/',
              'local': '_:',
              'prov': 'http://www.w3.org/ns/prov#',
              'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
              'schema': 'https://schema.org/',
              'units': 'http://w3id.org/nfdi4ing/metadata4ing#hasUnit'},
 '@id': 'N7d9290f5c1034acca4aa3aaeabb2fade',
 '@type': 'http://schema.org/SoftwareSourceCode',
 'http://schema.org/softwareVersion': '1.2.3a1'}


In [2]:
with h5tbx.File(mode='w') as h5:
    h5.create_dataset('test_dataset', shape=(3, ))
    grp = h5.create_group('grp')
    sub_grp = grp.create_group('Fan')
    sub_grp.create_dataset('D3', data=300)
    sub_grp['D3'].attrs['units', 'http://w3id.org/nfdi4ing/metadata4ing#hasUnits'] = 'mm'
    sub_grp['D3'].rdf['units'].object = 'https://qudt.org/vocab/unit/MilliM'
    sub_grp['D3'].attrs['standard_name', 'https://matthiasprobst.github.io/ssno/#standard_name'] = 'blade_diameter3'
    h5.dump(False)

## Dump the semantic metadata to JSON-LD format

The semantic metadata is stored in the RDF dictionaries of the HDF5 file, which the `h5rdmtoolbox`  can work with. Call `h5tbx.jsonld.dumps()` to extract it:

In [3]:
print(h5tbx.jsonld.dumps(h5.hdf_filename,
                         indent=2,
                         context={'schema': 'http://schema.org/',
                                  "ssno":  "https://matthiasprobst.github.io/ssno/#",
                                  "m4i": "http://w3id.org/nfdi4ing/metadata4ing#"}))

{
  "@context": {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
    "prov": "http://www.w3.org/ns/prov#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "https://schema.org/",
    "ssno": "https://matthiasprobst.github.io/ssno/#"
  },
  "@graph": [
    {
      "@id": "N332449db4f3b41b1a3127a6f37cfc8bb",
      "@type": "http://schema.org/SoftwareSourceCode",
      "http://schema.org/softwareVersion": "1.2.3a1"
    },
    {
      "@id": "N3f4ecefcd17c4ae18c4b8bd245b278f1",
      "@type": "m4i:NumericalVariable",
      "m4i:hasUnits": "mm",
      "ssno:standard_name": "blade_diameter3"
    }
  ]
}


## Dump the structural metadata to JSON-LD format

The structural or organizational metadata is the HDF5 internal layout, like groups, datasets, attributes and their properties including their relations:

In [4]:
hdf_jsonld = h5tbx.dump_jsonld(h5.hdf_filename, skipND=None)
print(hdf_jsonld)

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "hdf5": "http://purl.allotrope.org/ontologies/hdf5/1.8#"
    },
    "@type": "hdf5:File",
    "hdf5:rootGroup": {
        "@type": "hdf5:Group",
        "hdf5:attribute": [],
        "hdf5:name": "/",
        "hdf5:member": [
            {
                "@type": "hdf5:Group",
                "hdf5:name": "/grp",
                "hdf5:member": [
                    {
                        "@type": "hdf5:Group",
                        "hdf5:name": "/grp/Fan",
                        "hdf5:member": [
                            {
                                "@type": "hdf5:Dataset",
                                "hdf5:attribute": [
                                    {
                                        "@type": "hdf5:Attribute",
                                        "hdf5:name": "standard_name",
                                        "hdf

## Query the HDF-JSONLD file

The obtained JSON-LD file can be used to search for specific information. In the below example, all datasets with their sizes are extracted:

In [5]:
sparql_query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX hdf5: <http://purl.allotrope.org/ontologies/hdf5/1.8#>

SELECT ?ds_name ?ds_size
WHERE {
    ?group rdf:type hdf5:Dataset .
    ?group hdf5:name ?ds_name .
    ?group hdf5:size ?ds_size .
}
"""

In [6]:
import rdflib
g = rdflib.Graph()
g.parse(data=hdf_jsonld, format='json-ld')
results = g.query(sparql_query)

In [7]:
for b in results.bindings:
    print(b)

{rdflib.term.Variable('ds_name'): rdflib.term.Literal('/grp/Fan/D3'), rdflib.term.Variable('ds_size'): rdflib.term.Literal('1')}
{rdflib.term.Variable('ds_name'): rdflib.term.Literal('/test_dataset'), rdflib.term.Variable('ds_size'): rdflib.term.Literal('3')}


In [8]:
# convert results to dataframe:
import pandas as pd
df = pd.DataFrame(results.bindings)
df

,ds_name,ds_size
0,/grp/Fan/D3,1
1,/test_dataset,3
